This notebook parses sgf files generated by selfplay (or victimplay).

It presents the files in a searchable and filterable way,
and allows one to easily export files for viewing in Sabaki.

### Load libraries

In [ ]:
import dataclasses
import os
import posix
import random
import re
import sys
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

In [ ]:
sys.path.append("../engines/KataGo-custom/python")

from sgfmill import sgf

### Load games

In [ ]:
# DATA_DIR = "/nas/ucb/tony/go-attack/training/emcts1-v2/cp79-vis1"
# DATA_DIR = "/nas/ucb/tony/go-attack/training/emcts1-v2/cp79-vis1-apt"
# DATA_DIR = "/nas/ucb/tony/go-attack/training/emcts1-v2/cp99-vis1"
DATA_DIR = "/nas/ucb/tony/go-attack/training/emcts1-v2/cp127-vis1"

SELFPLAY_DIR = f"{DATA_DIR}/selfplay"
RAW_SGF_STRS: List[str] = []

bot_de: posix.DirEntry
for bot_de in tqdm([de for de in os.scandir(SELFPLAY_DIR) if de.is_dir()]):

    sgfs_path = os.path.join(bot_de.path, "sgfs")
    for sgf_de in [de for de in os.scandir(sgfs_path) if de.is_file()]:
        sgf_path: str = sgf_de.path
        with open(sgf_path) as f:
            for line in f.readlines():
                RAW_SGF_STRS.append(line.strip())

len(RAW_SGF_STRS)

### Parse games

In [ ]:
@dataclasses.dataclass
class GameInfo:
    board_size: int
    gtype: str
    start_turn_idx: int
    init_turn_num: int
    used_initial_position: bool

    victim_color: str
    adv_color: str
    win_color: str

    adv_name: str
    adv_steps: int
    adv_win: bool
    adv_minus_victim_score: float  # With komi
    adv_minus_victim_score_wo_komi: float  # No komi

    komi: float  # Positive if white has the advantage
    adv_komi: float  # Positive if adv has the advantage

    # Number of extra stones black places at start of game,
    # equivalent to the number of white passes at start of game.
    handicap: int
    has_handicap: bool  # Whether handicap is nonzero

    is_continuation: bool  # Whether game is continuation of previous game

    # Total number of moves (including passes)
    num_moves: int

    # How many times each player passed
    num_b_pass: int
    num_w_pass: int
    num_adv_pass: int  # Number of time adversary passed in the game
    num_victim_pass: int  # Number of times victim passed in the game

    ko_rule: str
    score_rule: str
    tax_rule: str
    sui_legal: bool
    has_button: bool
    whb: str  # whiteHandicapBonus
    fpok: bool  # friendly pass ok

    sgf_str: str  # raw sgf string


def get_game_info(sgf_str: str) -> GameInfo:
    sgf_game = sgf.Sgf_game.from_string(sgf_str)

    b_name = sgf_game.get_player_name("b")
    w_name = sgf_game.get_player_name("w")

    victim_color = {b_name: "b", w_name: "w"}["victim"]
    adv_color = {"b": "w", "w": "b"}[victim_color]

    adv_raw_name = {"b": b_name, "w": w_name}[adv_color]
    adv_name = (
        adv_raw_name.split("__victim")[0]
        if adv_color == "b"
        else adv_raw_name.split("victim__")[1]
    )
    adv_steps = (
        0
        if adv_name == "random"
        else int(re.search(r"\-s([0-9]+)\-", adv_name).group(1))
    )

    win_color = sgf_game.get_winner()
    lose_color = {"b": "w", "w": "b", None: None}[win_color]

    if win_color is None:
        adv_minus_victim_score = 0
    else:
        win_score = float(sgf_game.get_root().get("RE").split("+")[1])
        adv_minus_victim_score = {
            win_color: win_score,
            lose_color: -win_score,
        }[adv_color]

    komi = sgf_game.get_komi()
    adv_komi = {"w": komi, "b": -komi}[adv_color]

    handicap = int(sgf_game.root.get("HA"))

    def comment_prop(
        prop_name: str,
        default=None,
    ) -> str:
        comments = sgf_game.root.get("C")
        if prop_name not in comments:
            return default
        return comments.split(f"{prop_name}=")[1].split(",")[0]

    rule_str = sgf_game.root.get("RU")

    num_b_pass: int = sum(
        node.get_move() == ("b", None) for node in sgf_game.get_main_sequence()
    )
    num_w_pass: int = sum(
        node.get_move() == ("w", None) for node in sgf_game.get_main_sequence()
    )

    return GameInfo(
        board_size=sgf_game.get_size(),
        gtype=comment_prop("gtype"),
        start_turn_idx=int(comment_prop("startTurnIdx")),
        init_turn_num=int(comment_prop("initTurnNum")),
        used_initial_position=comment_prop("usedInitialPosition") == "1",
        victim_color=victim_color,
        adv_color=adv_color,
        win_color=win_color,
        adv_name=adv_name,
        adv_steps=adv_steps,
        adv_win=adv_color == win_color,
        adv_minus_victim_score=adv_minus_victim_score,
        adv_minus_victim_score_wo_komi=adv_minus_victim_score - adv_komi,
        komi=komi,
        adv_komi=adv_komi,
        handicap=handicap,
        has_handicap=handicap > 0,
        is_continuation=sgf_game.get_root().has_setup_stones(),
        num_moves=len(sgf_game.get_main_sequence()) - 1,
        num_b_pass=num_b_pass,
        num_w_pass=num_w_pass,
        num_adv_pass={"b": num_b_pass, "w": num_w_pass}[adv_color],
        num_victim_pass={"b": num_b_pass, "w": num_w_pass}[victim_color],
        sgf_str=sgf_str,
        ko_rule=re.search(r"ko([A-Z]+)", rule_str).group(1),
        score_rule=re.search(r"score([A-Z]+)", rule_str).group(1),
        tax_rule=re.search(r"tax([A-Z]+)", rule_str).group(1),
        sui_legal=re.search(r"sui([0-9])", rule_str).group(1) == "1",
        has_button="button1" in rule_str,
        whb=re.search(r"whb([A-Z0-9\-]+)", rule_str).group(1)
        if "whb" in rule_str
        else "0",
        fpok="fpok" in rule_str,
    )


random.seed(42)
GAME_INFOS: List[GameInfo] = process_map(
    get_game_info,
    RAW_SGF_STRS,
    max_workers=64,
    chunksize=50,
)


In [ ]:
%%time
df = pd.DataFrame(
    [dataclasses.asdict(gi) for gi in GAME_INFOS]
)
print("gtypes:", df.gtype.unique())
print("Number of cleanup games:", (df.gtype == "cleanuptraining").sum())

# Filter to only normal games
df = df[(df.gtype == 'normal')]
print("Fraction continuation:    ", df.is_continuation.mean())
print("Fraction used_initial_pos:", df.used_initial_position.mean())
print("max(init_turn_num)       :", df.init_turn_num.max())

df.head()

In [ ]:
df19 = df[df.board_size == 19]
df19.head()

### Analyze data

In [ ]:
def hist_3d(
    df: pd.DataFrame,
    t_key: str,
    ts: np.ndarray,
    v_key: str,
    ax: plt.Axes,
    cmap: plt.cm.ScalarMappable = plt.cm.inferno,
    bins: int = 50,
):
    """Adapted from https://stackoverflow.com/a/65126279/1337463"""
    ts = np.sort(ts)

    #calculate now the histogram and plot it for each column
    for i, t in enumerate(ts):
        #extract the current column from your df by its number
        col = df[df[t_key] == t][v_key]

        #determine the histogram values, here you have to adapt it to your needs
        histvals, edges = np.histogram(col, bins=bins, density=True)

        #calculate the center and width of each bar
        #obviously not necessary to do this for each column if you always have the same bins 
        #but if you choose for np.histogram other parameters, the bins may not be the same for each histogram
        xcenter = np.convolve(edges, np.ones(2), "valid")/2
        xwidth = np.diff(edges)

        #plot the histogram as a bar for each bin
        #now with continuous color mapping and edgecolor, so we can better see all bars
        ax.bar(left=xcenter, height=histvals, width=xwidth, zs=t, zdir="y", color=cmap(1-i/len(ts)), alpha=0.666, edgecolor="grey")

    ax.set_xlabel(v_key)
    ax.set_ylabel(t_key);
    ax.set_zlabel("density")

    ax.set_ylim(ts[-1],ts[0])
    ax.set_yticks(ts[::10])

In [ ]:
plt.figure(figsize=(12, 18), dpi=200)
plt.suptitle(SELFPLAY_DIR, fontsize=16, y=0.92);

plt.subplot(3, 2, 1)
df19[df19.adv_color == "b"].groupby("adv_steps").mean().adv_win.plot(label="adv = black")
df19[df19.adv_color == "w"].groupby("adv_steps").mean().adv_win.plot(label="adv = white")
plt.ylabel("Win rate"); plt.title("19x19 win rate")
plt.legend()

plt.subplot(3, 2, 2)
df19[df19.adv_color == "b"].groupby("adv_steps").count().adv_win.plot(label="adv = black")
df19[df19.adv_color == "w"].groupby("adv_steps").count().adv_win.plot(label="adv = white")
plt.ylabel("# of games"); plt.title("19x19 game count")
plt.legend()

ax = plt.subplot(3, 2, 3, projection="3d")
hist_3d(df19, ts=np.sort(df19.adv_steps.unique())[:-3], t_key="adv_steps", v_key="adv_minus_victim_score", ax=ax)
ax.invert_xaxis()
plt.title("19x19 score evolution")

ax = plt.subplot(3, 2, 4, projection="3d")
hist_3d(df19, ts=np.sort(df19.adv_steps.unique())[:-3], t_key="adv_steps", v_key="num_moves", ax=ax)
plt.title("19x19 # of moves evolution")

plt.subplot(3, 2, 5)
df.board_size.hist()
plt.title("Board size distribution");

plt.subplot(3, 2, 6)
df19.adv_komi.hist(bins=100);
plt.title("Komi distribution");

### Games

In [ ]:
def sgf_str_to_url(sgf_str: str) -> str:
    return f"https://humancompatibleai.github.io/sgf-viewer/#sgf={sgf_str}"

@dataclasses.dataclass(frozen=True)
class Url:
    """Wrapper around a URL string to provide nice display in IPython environments."""

    url: str
    text: str

    def _repr_html_(self):
        """HTML link to this URL."""
        return f'<a href="{self.url}">{self.text}</a>'

    def __str__(self):
        """Return the underlying string."""
        return self.url

In [ ]:
ADV_STEP_VIZ = list(sorted(df19.adv_steps.unique()))[-5]
print(ADV_STEP_VIZ)
df_win = df19[df19.adv_steps == ADV_STEP_VIZ]

In [ ]:
Url(
    url=sgf_str_to_url(df_win[df_win.adv_color == "b"].sgf_str.iloc[10]),
    text="b_win1",
)

In [ ]:
Url(
    url=sgf_str_to_url(df_win[df_win.adv_color == "b"].sgf_str.iloc[11]),
    text="b_win2",
)

In [ ]:
Url(
    url=sgf_str_to_url(df_win[df_win.adv_color == "w"].sgf_str.iloc[10]),
    text="w_win1",
)

In [ ]:
Url(
    url=sgf_str_to_url(df_win[df_win.adv_color == "w"].sgf_str.iloc[11]),
    text="w_win2",
)